# Payoff d'un condor call sur SPY

TP sur la structure condor (long K1, short K2, short K3, long K4 avec K1 < K2 < K3 < K4).
Objectifs :
1. Fixer un spot de référence via l'historique SPY.
2. Visualiser un payoff en "plateau" central, borné en gain et en perte.
3. Illustrer l'effet d'écarter les ailes (K1/K4) sur la largeur du plateau.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from pathlib import Path

plt.style.use("seaborn-v0_8-darkgrid")

def fetch_spy_history(period="1y", interval="1d") -> pd.Series:
    """Récupère les prix SPY (cache local puis fallback yfinance)."""
    cache_path = Path("notebooks/GPT/_cache_spy_close.csv")
    if cache_path.exists():
        try:
            cached = pd.read_csv(cache_path, index_col=0, parse_dates=True)
            if not cached.empty and "Close" in cached.columns:
                return cached["Close"]
        except Exception:
            pass
    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    close = data["Close"]
    try:
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        close.to_csv(cache_path, index_label="date")
    except Exception:
        pass
    return close

# Récupération des prix de clôture (dernier close pour spot_ref)
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])



def payoff_call(spot: float, strike: float) -> float:
    return max(spot - strike, 0.0)

def payoff_condor(spot: float, k1: float, k2: float, k3: float, k4: float) -> float:
    return (
        payoff_call(spot, k1)
        - payoff_call(spot, k2)
        - payoff_call(spot, k3)
        + payoff_call(spot, k4)
    )

    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    return data["Close"]

k1 = spot_ref * 0.9
k2 = spot_ref * 0.97
k3 = spot_ref * 1.03
k4 = spot_ref * 1.1


## Évolution du sous-jacent (SPY)
Clôtures sur un an et repère du spot de référence (dernier close).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()

## Payoff du condor call long
Plateau de gain entre K2 et K3, pertes bornées en dehors de K1/K4.

In [ ]:
S_grid = np.linspace(spot_ref * 0.5, spot_ref * 1.5, 200)
payoffs = [payoff_condor(S, k1, k2, k3, k4) for S in S_grid]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(S_grid, payoffs, color="navy", label="Payoff condor long")
for k, style, label in [
    (k1, ":", f"K1 = {k1:.2f}"),
    (k2, "--", f"K2 = {k2:.2f}"),
    (k3, "--", f"K3 = {k3:.2f}"),
    (k4, ":", f"K4 = {k4:.2f}"),
]:
    ax.axvline(k, color="gray", linestyle=style, label=label)
ax.set_xlabel("Spot S")
ax.set_ylabel("Payoff à maturité")
ax.set_title("Condor call sur SPY : payoff vs spot")
ax.legend()
plt.show()

In [ ]:

import ipywidgets as widgets
from IPython.display import display, Markdown

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='Spot')
slider_k1 = widgets.FloatSlider(value=spot_ref*0.9, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K1')
slider_k2 = widgets.FloatSlider(value=spot_ref*0.95, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K2')
slider_k3 = widgets.FloatSlider(value=spot_ref*1.05, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K3')
slider_k4 = widgets.FloatSlider(value=spot_ref*1.1, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K4')
output = widgets.Output()

def _update_payoff(change=None):
    with output:
        output.clear_output()
        s = spot_slider.value
        k1 = slider_k1.value
        k2 = slider_k2.value
        k3 = slider_k3.value
        k4 = slider_k4.value
        p = payoff_condor(s, k1, k2, k3, k4)
        display(Markdown(f"**Spot = {s:.2f}**

- K1 = {k1:.2f}
- K2 = {k2:.2f}
- K3 = {k3:.2f}
- K4 = {k4:.2f}
- Payoff condor = {p:.4f}"))

for sl in (spot_slider, slider_k1, slider_k2, slider_k3, slider_k4):
    sl.observe(_update_payoff, names='value')

_update_payoff()
display(widgets.VBox([spot_slider, slider_k1, slider_k2, slider_k3, slider_k4, output]))
